In [191]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
import _pickle
import datetime
import pandas as pd

In [172]:
# _pickle import
with open('pkls/zansa_12.pkl', 'rb') as f:
    df = _pickle.load(f)
with open('pkls/mesh12_list.pkl', 'rb') as f:
    mesh12_list = _pickle.load(f)
with open('pkls/fm_list.pkl', 'rb') as f:
    fm_list = _pickle.load(f)
with open('pkls/tsdiv_list.pkl', 'rb') as f:
    tsdiv_list = _pickle.load(f)
with open('pkls/kind_list.pkl', 'rb') as f:
    kind_list = _pickle.load(f)
with open('pkls/condition_list.pkl', 'rb') as f:
    condition_list = _pickle.load(f)
with open('pkls/not_condition_list.pkl', 'rb') as f:
    ncondition_list = _pickle.load(f)
with open('pkls/other_index.pkl', 'rb') as f:
    others_index = _pickle.load(f)
with open('pkls/mesh_list.pkl', 'rb') as f:
    mesh_list = _pickle.load(f)

In [173]:
class TF(nn.Module):
    def __init__(self, input_size, R):
        super(TF, self).__init__()
        self.I = input_size[0]
        self.J = input_size[1]
        self.K = input_size[2]
        self.U = nn.Parameter(torch.Tensor(R, self.I).uniform_(0,1), requires_grad=True)
        self.V = nn.Parameter(torch.Tensor(R, self.J).uniform_(0,1), requires_grad=True)
        self.W = nn.Parameter(torch.Tensor(R, self.K).uniform_(0,1), requires_grad=True)
        
    def non_negative(self, R=None):
#         torch.clamp(self.U, min=0)
#         torch.clamp(self.V, min=0)
#         torch.clamp(self.W, min=0)
        torch.clamp(self.U, min=0)
        torch.clamp(self.V, min=0)
        torch.clamp(self.W, min=0)

    def forward_one_rank(self, u, v, w):
        UV = torch.ger(u, v)
        UV2 = UV.unsqueeze(2).repeat(1,1,self.K)
        W2 = w.unsqueeze(0).unsqueeze(1).repeat(self.I, self.J, 1)
        outputs = UV2 * W2
        return outputs
    
    def forward(self, X=None):
        if X is not None:
            weight = (X != 0).float()
        else:
            weight = Variable(torch.ones(self.I, self.J, self.K))
        
        output = self.forward_one_rank(self.U[0], self.V[0], self.W[0])
        
        for i in np.arange(1, R):
            one_rank = self.forward_one_rank(self.U[i], self.V[i], self.W[i])
            output = output + one_rank
        
        return output

In [282]:
# X = torch.Tensor(28, 62, 355).uniform_(0, 1)
# print(X[0,27,1])

0.14438432455062866


In [295]:
def mse_loss(output, df, tslist, meshlist, condlist, ncondlist, othlist):
    df1 = df.loc[othlist]
    bar = '-'
    loss = 0
    loss_all = 0
    for i in range(len(condlist)):
        print(bar)
        bar += '-'
        if ncondlist[i] == '':
            df2 = df.loc[list(df[df.text.str.contains(condlist[i])].index)]
        elif ncondlist[i] != '':
            a = list(df[df.text.str.contains(condlist[i])].index)
            b = list(df[df.text.str.contains(ncondlist[i])].index)
            c = list(set(a) - set(b))
            df2 = df.loc[c]
            
        for j in range(len(tslist)):
            ts_condition_from = df2.ts >= tslist[j]
            if j != 61:
                ts_condition_to = df2.ts < tslist[j+1]
                
            for k in range(len(meshlist)):
                mesh_condition = df2.mesh12 == meshlist[k]
                if k == len(tslist):
                    val = len(df2[(ts_condition_from) & (mesh_condition)].index)
                else:
                    val = len(df2[(ts_condition_from) & (ts_condition_to) & (mesh_condition)].index)
                loss = output[i, j, k] - val
                loss_all += loss
                loss = 0
                val = 0
                
            if i == len(tslist):
                val = len(df2[(ts_condition_from) & (mesh_condition)].index)
            else:
                val = len(df2[(ts_condition_from) & (ts_condition_to) & (mesh_condition)].index)
            loss = output[27,j,k] - val
            loss_all += loss
            loss = 0
            val = 0
        print(loss_all)

In [296]:
mse_loss(X, df, tsdiv_list, mesh_list, condition_list, ncondition_list, others_index)

-
10722.361548542976
--
21233.872035205364
---
32204.322182893753
----
43193.87131947279
-----
54091.60136860609
------
64974.933781564236
-------
75936.55491298437
--------
86980.40790659189
---------
98053.59385573864
----------
109026.72228419781
-----------
120116.67225444317
------------
131103.04897111654
-------------
142094.04866331816
--------------
153166.78853029013
---------------
164198.03318929672
----------------
175191.31732922792
-----------------
186165.6138856411
------------------
196978.48128855228
-------------------
207960.86993294954
--------------------
218947.20431232452
---------------------
230007.84637647867
----------------------
241000.60425937176
-----------------------
252073.77873945236
------------------------
263143.36879736185
-------------------------
274088.55863058567
--------------------------
284921.51477241516
---------------------------
295970.85495889187


In [ ]:
input_size = (28, 62, 355)
# X = torch.Tensor(62, 355, 28).uniform_(0, 1)

R = 2
model = TF(input_size, R)
# criterion = torch.nn.MSELoss(size_average=False)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# X = Variable(X)

epoch = 1000
for i in range(epoch):
    output = model.forward()
    loss = mse_loss(output, df, tsdiv_list, mesh_list, condition_list, ncondition_list, others_index)
    
    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()
    
    model.non_negative()
    
    if i % 100 == 0:
        print('{}: {}'.format(i, loss.data[0]))
        
print('{}: {}'.format(epoch, loss.data[0]))
print(model.forward().shape)

-
Variable containing:
 1597.0560
[torch.FloatTensor of size 1]

--
Variable containing:
 7393.3276
[torch.FloatTensor of size 1]

---
Variable containing:
 17895.9648
[torch.FloatTensor of size 1]

----
Variable containing:
 22860.8633
[torch.FloatTensor of size 1]

-----
Variable containing:
 27086.2109
[torch.FloatTensor of size 1]

------
Variable containing:
 31909.5996
[torch.FloatTensor of size 1]

-------
Variable containing:
 37841.4609
[torch.FloatTensor of size 1]

--------
Variable containing:
 42794.9414
[torch.FloatTensor of size 1]

---------
Variable containing:
 46053.4414
[torch.FloatTensor of size 1]

----------
Variable containing:
 51467.2812
[torch.FloatTensor of size 1]

-----------
Variable containing:
 58678.7031
[torch.FloatTensor of size 1]

------------
Variable containing:
 65161.9492
[torch.FloatTensor of size 1]

-------------
Variable containing:
 73416.0547
[torch.FloatTensor of size 1]

--------------
Variable containing:
 80672.0156
[torch.FloatTensor

In [9]:
model.non_negative()
print(model.U.shape, model.V.shape, model.W.shape)

torch.Size([2, 62]) torch.Size([2, 28]) torch.Size([2, 352])


In [ ]:
c = condition_list[1]
print(c)
print(type(c))
xlist = list(df[df.text.str.contains(c)].index)
print(xlist)
df1 = df.loc[xlist]
print(df1)
df1[(df1.ts >= pd.to_datetime('20161201')) & (df1.ts < pd.to_datetime('20170826')) & (df1.mesh12 == 533903731)]

In [ ]:
c = 0
nc = 0
for i in mesh_list:
    for j in range(len(df.index)):
        if df.iloc[j, 5] == i:
            c += 1
        else:
            nc += 1
    print(c,nc)
            